# M5 Forecasting Accuracy

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
pd.options.display.max_columns = 50
import matplotlib.pyplot as plt
import seaborn as sns
from  datetime import datetime, timedelta

import os
import gc
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [ ]:
# データの取得, DataFrame化
path = "../input/m5-forecasting-accuracy"

sales = pd.read_csv(os.path.join(path, "sales_train_evaluation.csv"))
calendar = pd.read_csv(os.path.join(path, "calendar.csv"))
prices = pd.read_csv(os.path.join(path, "sell_prices.csv"))
sample_submission = pd.read_csv(os.path.join(path, "sample_submission.csv"))

# 回帰する日付の追加
for d in range(1942,1970):
    col = 'd_' + str(d)
    sales[col] = 0
    sales[col] = sales[col].astype(np.int16)


In [ ]:
# メモリ消費を抑えるために, ダウンキャストする　(kaggle notebook上でデータを整理する際にRAMを消費尽くしてしまう)
# 最小値, 最大値が収まるデータ型に変更する
def downcast(df):
    cols = df.dtypes.index.tolist() 
    types = df.dtypes.values.tolist()
    for i,t in enumerate(types):
        if 'int' in str(t):
            if df[cols[i]].min() > np.iinfo(np.int8).min and df[cols[i]].max() < np.iinfo(np.int8).max:
                df[cols[i]] = df[cols[i]].astype(np.int8)
            elif df[cols[i]].min() > np.iinfo(np.int16).min and df[cols[i]].max() < np.iinfo(np.int16).max:
                df[cols[i]] = df[cols[i]].astype(np.int16)
            elif df[cols[i]].min() > np.iinfo(np.int32).min and df[cols[i]].max() < np.iinfo(np.int32).max:
                df[cols[i]] = df[cols[i]].astype(np.int32)
            else:
                df[cols[i]] = df[cols[i]].astype(np.int64)
        elif 'float' in str(t):
            if df[cols[i]].min() > np.finfo(np.float16).min and df[cols[i]].max() < np.finfo(np.float16).max:
                df[cols[i]] = df[cols[i]].astype(np.float16)
            elif df[cols[i]].min() > np.finfo(np.float32).min and df[cols[i]].max() < np.finfo(np.float32).max:
                df[cols[i]] = df[cols[i]].astype(np.float32)
            else:
                df[cols[i]] = df[cols[i]].astype(np.float64)
        elif t == np.object:
            if cols[i] == 'date':
                df[cols[i]] = pd.to_datetime(df[cols[i]], format='%Y-%m-%d')
            else:
                df[cols[i]] = df[cols[i]].astype('category')
    return df  

sales = downcast(sales)
prices = downcast(prices)
calendar = downcast(calendar)

# データ整理 meltメソッドを用いてsalesのcolumn'd_〇'を'd'columnにまとめ, calender.csvと'd' columnでmergeできるようにする
df = pd.melt(sales, id_vars=['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], var_name='d', value_name='sold').dropna()
df = pd.merge(df, calendar, on='d', how='left')
df = pd.merge(df, prices, on=['store_id','item_id','wm_yr_wk'], how='left') 
display(df.head())
print(df.info())

Number of Index : 59181090 (= 30980(商品の数) * 1941(日数))  

# EDA  

### sell-priceについて

In [ ]:
# 特定の商品において, priceとsoldの関係性を簡単に確認 soldとpriceの相関係数も計算
itemID =['HOBBIES_1_001','HOBBIES_1_010','HOBBIES_2_001','HOBBIES_2_100',
        'HOUSEHOLD_1_001','HOUSEHOLD_1_010','HOUSEHOLD_2_001','HOUSEHOLD_2_100',
        'FOODS_1_001','FOODS_1_010','FOODS_2_001','FOODS_2_100']
print('赤 : sold / 青 : price')
print('CA_1')
r,c=0,0
fig, ax = plt.subplots(3,4,figsize=(18, 12))
for i in range(0,12):
    df_store = df[df['store_id']=='CA_1']
    item_df = df_store[df_store['item_id']==itemID[i]]
    print('{} sold/price相関係数:'.format(itemID[i]),item_df['sold'].corr(item_df['sell_price']))
    ax2 = ax[r][c].twinx() 
    ax[r][c].plot(item_df['date'],item_df['sell_price'],color='b',alpha=0.5)
    ax2.plot(item_df['date'],item_df['sold'],color='r',alpha=0.5)
    ax[r][c].set_title(itemID[i])
    c+=1
    if c ==4:
        c=0
        r+=1
#plt.savefig('CA_1_price.png')
plt.show()

print('\nCA_2')
r,c=0,0
fig, ax = plt.subplots(3,4,figsize=(18, 12))
for i in range(0,12):
    df_store = df[df['store_id']=='CA_2']
    item_df = df_store[df_store['item_id']==itemID[i]]
    print('{} sold/price相関係数:'.format(itemID[i]),item_df['sold'].corr(item_df['sell_price']))
    ax2 = ax[r][c].twinx() 
    ax[r][c].plot(item_df['date'],item_df['sell_price'],color='b',alpha=0.5)
    ax2.plot(item_df['date'],item_df['sold'],color='r',alpha=0.5)
    ax[r][c].set_title(itemID[i])
    c+=1
    if c ==4:
        c=0
        r+=1
#plt.savefig('CA_2_price.png')
plt.show()

print('\nCA_3')
r,c=0,0
fig, ax = plt.subplots(3,4,figsize=(18, 12))
for i in range(0,12):
    df_store = df[df['store_id']=='CA_3']
    item_df = df_store[df_store['item_id']==itemID[i]]
    print('{} sold/price相関係数:'.format(itemID[i]),item_df['sold'].corr(item_df['sell_price']))
    ax2 = ax[r][c].twinx() 
    ax[r][c].plot(item_df['date'],item_df['sell_price'],color='b',alpha=0.5)
    ax2.plot(item_df['date'],item_df['sold'],color='r',alpha=0.5)
    ax[r][c].set_title(itemID[i])
    c+=1
    if c ==4:
        c=0
        r+=1
#plt.savefig('CA_3_price.png')
plt.show()

print('\nTX_1')
r,c=0,0
fig, ax = plt.subplots(3,4,figsize=(18, 12))
for i in range(0,12):
    df_store = df[df['store_id']=='TX_1']
    item_df = df_store[df_store['item_id']==itemID[i]]
    print('{} sold/price相関係数:'.format(itemID[i]),item_df['sold'].corr(item_df['sell_price']))
    ax2 = ax[r][c].twinx() 
    ax[r][c].plot(item_df['date'],item_df['sell_price'],color='b',alpha=0.5)
    ax2.plot(item_df['date'],item_df['sold'],color='r',alpha=0.5)
    ax[r][c].set_title(itemID[i])
    c+=1
    if c ==4:
        c=0
        r+=1
#plt.savefig('TX_1_price.png')
plt.show()

print('\nWI_1')
r,c=0,0
fig, ax = plt.subplots(3,4,figsize=(18, 12))
for i in range(0,12):
    df_store = df[df['store_id']=='WI_1']
    item_df = df_store[df_store['item_id']==itemID[i]]
    print('{} sold/price相関係数:'.format(itemID[i]),item_df['sold'].corr(item_df['sell_price']))
    ax2 = ax[r][c].twinx() 
    ax[r][c].plot(item_df['date'],item_df['sell_price'],color='b',alpha=0.5)
    ax2.plot(item_df['date'],item_df['sold'],color='r',alpha=0.5)
    ax[r][c].set_title(itemID[i])
    c+=1
    if c ==4:
        c=0
        r+=1
#plt.savefig('WI_1_price.png')
plt.show()


・priceの変動は少なく, 利用する期間内において周期性は確認できない  
・相関係数はsoldとpriceの変動間隔が異なるため参考にしづらいが, 図を確認する限りでは商品の価格変動と, 販売個数に相関はない

### soldについて

In [ ]:
# 例として一つの商品のsoldの推移を確認する
df_1 = df[df['item_id']=='HOBBIES_1_001']
plt.plot(df_1['date'],df_1['sold'])
plt.title('HOBBIES_1_001')
plt.xlabel('date')
plt.ylabel('sold')
#plt.savefig('HOBBIES_1_001_sold_.png')
plt.show()

In [ ]:
# soldの分布を確認する
state_id_lst = df['state_id'].unique()
store_id_lst = df['store_id'].unique()
category = df['cat_id'].unique()

fig, ax = plt.subplots(2,5,figsize=(20, 9))
r,c=0,0
for store_id in store_id_lst:
    df_store = df[df['store_id']==store_id]
    sns.boxplot(x='cat_id',y='sold',data=df_store,ax=ax[r][c])
    ax[r][c].set_title(store_id)
    c +=1
    if c==5:
        r+=1
        c=0
#plt.savefig('Dist_sold.png')
plt.show()

* 多くの商品のsoldは0であり, おそらく商品によって多数売れることが想定される
* 最大販売数においてFOODSが最も多く, HOUSEHOLD, HOBBIESはそれと比較して少ない傾向がある

In [ ]:

# 各州における全ての商品の販売数(総和)の推移を可視化
fig, ax = plt.subplots(1,3,figsize=(20, 4))
for i,state_id in enumerate(state_id_lst):
    df_state = df[df['state_id']==state_id]
    sold=df_state.groupby('date').sum()['sold']
    sold_rol = sold.rolling('30D',center=True).mean()
    ax[i].plot(sold,c='b') # 生データの推移：青
    ax[i].plot(sold_rol,c='r') # 移動平均：赤
    ax[i].set_ylim(0,26000)
    ax[i].set_title(state_id)
#plt.savefig('Each_state_all_goods_sold.png')
plt.show()
# 各州における全ての商品の販売数(総和) 週,月,年について可視化
fig, ax = plt.subplots(1,3,figsize=(20, 4))
for i,state_id in enumerate(state_id_lst):
    df_state = df[df['state_id']==state_id]
    df_state=df_state.groupby('date').sum()
    sns.boxplot(x='wday',y='sold',data=df_state,ax=ax[i])
    ax[i].set_title(state_id)
    ax[i].set_xticklabels(['Saturday', 'Sunday','Monday', 'Tuesday', 'Wendthday', 'Thursday', 'Fryday'])
#plt.savefig('Each_state_all_goods_sold_box_week.png')
plt.show()
fig, ax = plt.subplots(1,3,figsize=(20, 4))
for i,state_id in enumerate(state_id_lst):
    df_state = df[df['state_id']==state_id]
    df_state=df_state.groupby('date').sum()
    sns.boxplot(x='month',y='sold',data=df_state,ax=ax[i])
    ax[i].set_title(state_id)
    ax[i].set_xticklabels(['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec'])
#plt.savefig('Each_state_all_goods_sold_box_month.png')
plt.show()
fig, ax = plt.subplots(1,3,figsize=(20, 4))
for i,state_id in enumerate(state_id_lst):
    df_state = df[df['state_id']==state_id]
    df_state=df_state.groupby('date').sum()
    sns.boxplot(x='year',y='sold',data=df_state,ax=ax[i])
    ax[i].set_title(state_id)
    ax[i].set_xticklabels(['2011', '2012','2013', '2014', '2015', '2016'])
#plt.savefig('Each_state_all_goods_sold_box_year.png')
plt.show()


# 各お店における全ての商品の販売数(総和)の推移を可視化
r,c=0,0
fig, ax = plt.subplots(2,5,figsize=(30, 8))
for store_id in store_id_lst:
    df_store = df[df['store_id']==store_id]
    sold=df_store.groupby('date').sum()['sold']
    sold_rol = sold.rolling('30D',center=True).mean()
    ax[r][c].plot(sold,c='b') # 生データの推移：青
    ax[r][c].plot(sold_rol,c='r') # 移動平均：赤
    ax[r][c].set_ylim(0,9000)
    ax[r][c].set_title(store_id)
    c+=1
    if c ==5:
        c=0
        r+=1
#plt.savefig('Each_state_each_goods_sold_year.png')
plt.show()
        
# 全てのお店における全ての商品の販売数の推移を可視化
sold=df.groupby('date').sum()['sold']
sold_rol = sold.rolling('30D',center=True).mean()
plt.plot(sold,c='b') # 生データの推移：青
plt.plot(sold_rol,c='r') # 移動平均：赤
plt.title('all aggregate units sales')
#plt.savefig('all_aggregate_units_sales.png')
plt.show()

In [ ]:

# 各お店におけるカテゴリ毎の商品の販売数(総和)の推移を可視化
#　箱ひげ図によりデータの分布を可視化
r,c=0,0
for state_id in state_id_lst:
    fig, ax = plt.subplots(3,4,figsize=(25, 15))
    df_state = df[df['state_id']==state_id]
    for i,cat_id in enumerate(category):
        df_state_cat = df_state[df_state['cat_id']==cat_id]
        df_sc=df_state_cat.groupby('date').sum()
        sold_rol = df_sc['sold'].rolling('30D',center=True).mean()
        ax[i][0].plot(df_sc['sold'],c='b') # 生データの推移：青
        ax[i][0].plot(sold_rol,c='r') # 移動平均：赤
        ax[i][0].set_title(state_id+'_'+cat_id)
        sns.boxplot(x='wday', y='sold', data=df_sc,ax=ax[i][1])
        ax[i][1].set_title(state_id+'_'+cat_id+' week')
        ax[i][1].set_xticklabels(['Saturday', 'Sunday','Monday', 'Tuesday', 'Wendthday', 'Thursday', 'Fryday'])
        sns.boxplot(x='month', y='sold', data=df_sc,ax=ax[i][2])
        ax[i][2].set_title(state_id+'_'+cat_id+' month')
        ax[i][2].set_xticklabels(['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec'])
        sns.boxplot(x='year', y='sold', data=df_sc,ax=ax[i][3])
        ax[i][3].set_title(state_id+'_'+cat_id+' year')
        ax[i][3].set_xticklabels(['2011', '2012','2013', '2014', '2015', '2016'])
    #plt.savefig('{}_sold_box.png'.format(state_id))
    plt.show()
    


### sold について 可視化結果からの考察
* 全体, 州, お店, 商品のカテゴリ毎において商品販売数は全て右肩上がり
 * 年々増加が確認される
 * (移動平均の推移より, 年単位の周期性があるのではないか？)
* 州ごとに販売個数の推移が異なる
* 州が同一であってもお店によって総販売個数の推移の様子が異なる
* 州, 商品カテゴリにおいて曜日毎に販売個数が異なる
 * 土曜日曜が高く、水曜日まで減少し、木曜日から増加に転じる傾向(週のなかでの周期性)
 * 商品カテゴリにおいて, HOBBISEでは土曜＞日曜, HOUSEHOLD, FOODSでは日曜＞土曜となった
 
したがって、曜日、年の情報は回帰に特に有効であると考える

### SNAPの有無について
SNAP : 低所得の家族や個人に、食品を購入するための電子給付振替のデビットカードを提供するものである。多くの州では、月の10日間に渡って金銭的な給付が人々に分配され、それぞれの日に1/10の人々がカードで給付を受けることができる。
⇒　食品商品において販売数が増減する可能性

In [ ]:
# SNAPの有無による, FOODの販売数の変化を確認

# 各州における全ての商品の販売数(総和)の推移を可視化
fig, ax = plt.subplots(1,3,figsize=(20, 4))
for i,state_id in enumerate(state_id_lst):
    df_state = df[df['state_id']==state_id]
    df_state = df_state[df_state['cat_id']=='FOODS']
    sold=df_state.groupby('date').sum()['sold']
    sold_rol = sold.rolling('30D',center=True).mean()
    ax[i].plot(sold,c='c') # 生データの推移：シアン
    ax[i].plot(sold_rol,c='m') # 移動平均：マゼンタ
    
    df_state_snap = df_state[df_state['snap_{}'.format(state_id)]==1]
    df_ss=df_state_snap.groupby('date').sum()['sold']
    ss_sold_rol = df_ss.rolling('30D',center=True).mean()
    ax[i].plot(df_ss,c='b') # SNAP 生データの推移：青
    ax[i].plot(ss_sold_rol,c='r') # SNAP 移動平均：赤
    ax[i].set_ylim(0,16000)
    ax[i].set_title(state_id)
#plt.savefig('Each_state_SNAP_year.png')
plt.show()


# 各お店における全ての商品の販売数(総和)の推移を可視化
r,c=0,0
fig, ax = plt.subplots(2,5,figsize=(30, 8))
for store_id in store_id_lst:
    df_store = df[df['store_id']==store_id]
    df_state = df_state[df_state['cat_id']=='FOODS']
    sold=df_store.groupby('date').sum()['sold']
    sold_rol = sold.rolling('30D',center=True).mean()
    ax[r][c].plot(sold,c='c') # 生データの推移：シアン
    ax[r][c].plot(sold_rol,c='m') # 移動平均：マゼンタ
    
    df_store_snap = df_store[df_store['snap_{}'.format(store_id[:2])]==1]
    df_ss=df_store_snap.groupby('date').sum()['sold']
    ss_sold_rol = df_ss.rolling('30D',center=True).mean()
    ax[r][c].plot(df_ss,c='b') # SNAP 生データの推移：青
    ax[r][c].plot(ss_sold_rol,c='r') # SNAP 移動平均：赤
    ax[r][c].set_ylim(0,9000)
    ax[r][c].set_title(store_id)
    c+=1
    if c ==5:
        c=0
        r+=1
#plt.savefig('Each_store_SNAP_year.png')
plt.show()
        


In [ ]:
fig, ax = plt.subplots(3,3,figsize=(20, 12))
for i,state_id in enumerate(state_id_lst):
    df_state = df[df['state_id']==state_id]
    df_state = df_state[df_state['cat_id']=='FOODS']
    df_state_snap = df_state[df_state['snap_{}'.format(state_id)]==1]
    df_ss=df_state_snap.groupby('date').sum()

    sns.boxplot(x='wday', y='sold', data=df_ss,ax=ax[i][0])
    ax[i][0].set_title(state_id+'_'+'FOODS'+' week')
    ax[i][0].set_xticklabels(['Saturday', 'Sunday','Monday', 'Tuesday', 'Wendthday', 'Thursday', 'Fryday'])
    sns.boxplot(x='month', y='sold', data=df_ss,ax=ax[i][1])
    ax[i][1].set_title(state_id+'_'+'FOODS'+' month')
    ax[i][1].set_xticklabels(['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec'])
    sns.boxplot(x='year', y='sold', data=df_ss,ax=ax[i][2])
    ax[i][2].set_title(state_id+'_'+'FOODS'+' year')
    ax[i][2].set_xticklabels(['2011', '2012','2013', '2014', '2015', '2016'])
#plt.savefig('Each_store_goods_SNAP_year.png')
plt.show()

* お店ごとにSNAPの有無で若干の商品の販売数が上昇が確認できる  
 * WI_2, WI_3が顕著
* SNAPを無視した際の箱ひげ図と比較して, SNAPがある場合, 月ごとの販売数の変動が若干大きくなったになった印象

## 特徴量エンジニアリング

EDAで得られた情報から特徴量エンジニアリング(機械学習モデルに入力する特徴量の選択, 生成)を行う。 EDAより, 次の特徴量を追加することが有効であることが考えられる
* 一週間, 一月のラグ
* 移動平均  

したがって, 今回は存在するデータの最長1月先まで予測するため, 1月前の販売数とそれを基準として, 1日前, 2日前, 1週間前といったラグデータを用いる。
また, 全体的なトレンドとして商品販売数が同様な推移をしていることから, 周期的な推移が確認されるラグと, それぞれの移動平均が有効となることが考えられる。  
また, データの階層(CA, CA_1, CA_1_HOBBIESのような)ごとにトレンドが存在する可能性があるため, TargetEncodingを行う(Ordered Target Statistics)

In [ ]:
d_id = dict(zip(df.id.cat.codes, df.id))
d_store_id = dict(zip(df.store_id.cat.codes, df.store_id))

# 意味が重複しているデータの除去
df.drop(["date", "wm_yr_wk", "weekday"],axis=1,inplace=True)

# d(初日から何日目であるかを表すcolumn)をint型にする
df.d = df['d'].apply(lambda x: x.split('_')[1]).astype(np.int16)


In [ ]:

# カテゴリカル変数をエンコード
cols = df.dtypes.index.tolist()
types = df.dtypes.values.tolist()
for i,type in enumerate(types):
    if type.name == 'category':
        df[cols[i]] = df[cols[i]].cat.codes
        

# lag特徴の追加
# 予測する最大が28日後であるため, 28日前に対してlagを計算する
# 1,2,3日前, 1,2,4週間前, 約2月前のsoldを追加
lags = [1,2,3,7,14,28,56]
for lag in lags:
    df['sold_lag_'+str(lag)] = df.groupby(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'],as_index=False)['sold'].shift(lag+28).astype(np.float16)
    
# ラグにおける移動平均の追加
sold_lag_cols = ['sold_lag_7','sold_lag_28']
for win in [7,28] :
    for lag,lag_col in zip([7,28], sold_lag_cols):
        df['rmean_{}_{}'.format(lag,win)] = df.groupby(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'])['sold_lag_'+str(lag)].transform(lambda x: x.rolling(window=win).mean()).astype(np.float16)

# Target Encoding 
# sold に対して Orderd Target Encoding
cols = ['state_id','store_id','cat_id','dept_id']
new_cols = ['state_id_ots','store_id_ots','cat_id_ots','dept_id_ots']
df_new = pd.DataFrame()
import category_encoders as ce
te = ce.CatBoostEncoder(random_state=42)
for c,nc in zip(cols,new_cols):
    df[nc] = te.fit_transform(df[c],df['sold'])
df = downcast(df)        

# ラグの導入によるNanの除去
df = df[df['d']>=57]
df.tail()

In [ ]:
df.info()

In [ ]:
# 予測に利用する作成したデータの作成
df.to_pickle('data.pkl')
del df
gc.collect();

# Modeling and Forecast

In [ ]:
# 訓練, 検証用のデータの分割, テストデータを指定
data = pd.read_pickle('data.pkl')
valid = data[(data['d']>=1914) & (data['d']<1942)][['id','d','sold']]
test = data[data['d']>=1942][['id','d','sold']]
eval_preds = test['sold']
valid_preds = valid['sold']

In [ ]:
from lightgbm import LGBMRegressor
import joblib
import optuna
import sklearn.datasets

# optuna, lightgbmに用いるseed値
seed=42

def objectives(trial):
    # optunaでのハイパーパラメータサーチ範囲の設定
    params = {
            'learning_rate' : trial.suggest_float('learning_rate',0.05,0.1),
            'num_leaves': trial.suggest_int('num_leaves', 63, 255),
            'max_depth' : trial.suggest_int('max_depth', 4,8),
            'min_child_samples': trial.suggest_int('min_child_samples', 10, 100)
            }

    # LightGBMで学習+予測
    model = LGBMRegressor(random_state=seed,**params)# 追加部分
    model.fit(X_train, y_train,eval_set=[(X_train,y_train),(X_valid,y_valid)],eval_metric='tweedie',early_stopping_rounds=20,verbose=False)

    # 検証データを用いた評価
    score = model.score(X_valid, y_valid)
    
    return score

#store ごとにモデルを作成し, テストデータを予測する
stores = sales.store_id.cat.codes.unique().tolist()
for store in stores:
    df = data[data['store_id']==store]
    
    #訓練, 検証用, (テスト)　にデータを分割　
    X_train, y_train = df[df['d']<1914].drop('sold',axis=1), df[df['d']<1914]['sold']
    X_valid, y_valid = df[(df['d']>=1914) & (df['d']<1942)].drop('sold',axis=1), df[(df['d']>=1914) & (df['d']<1942)]['sold']
    X_test = df[df['d']>=1942].drop('sold',axis=1)
    
    print('*****Prediction for Store: {}*****'.format(d_store_id[store]))
    
    # optunaによる最適化呼び出し
    opt = optuna.create_study(direction='maximize',sampler=optuna.samplers.TPESampler(seed=seed))
    opt.optimize(objectives, n_trials=10)

    # 最適パラメータ取得
    trial = opt.best_trial
    params_best = dict(trial.params.items())
    params_best['random_seed'] = 0

    # 最適パラメータで学習/予測    
    model = LGBMRegressor(n_estimators=1000,random_state=seed,**params_best)
    
    model.fit(X_train, y_train, eval_set=[(X_train,y_train),(X_valid,y_valid)],
             eval_metric='tweedie', verbose=20, early_stopping_rounds=50)
    valid_preds[X_valid.index] = model.predict(X_valid)
    eval_preds[X_test.index] = model.predict(X_test)
    filename = 'model'+str(d_store_id[store])+'.pkl'
    # modelの保存
    joblib.dump(model, filename)
    del model, X_train, y_train, X_valid, y_valid
    gc.collect()

In [ ]:
# 訓練の際のvalidationの予測結果を獲得
valid['sold'] = valid_preds
validation = valid[['id','d','sold']]
validation = pd.pivot(validation, index='id', columns='d', values='sold').reset_index()
validation.columns=['id'] + ['F' + str(i + 1) for i in range(28)]
validation.id = validation.id.map(d_id).str.replace('evaluation','validation')

# testの予測結果を獲得
test['sold'] = eval_preds
evaluation = test[['id','d','sold']]
evaluation = pd.pivot(evaluation, index='id', columns='d', values='sold').reset_index()
evaluation.columns=['id'] + ['F' + str(i + 1) for i in range(28)]
#　カテゴリーIDをそれぞれのカテゴリーにリマップ
evaluation.id = evaluation.id.map(d_id)

#　submissionの作成 
submit = pd.concat([validation,evaluation]).reset_index(drop=True)
submit.to_csv('submission.csv',index=False)

参考 :    
https://www.kaggle.com/headsortails/back-to-predict-the-future-interactive-m5-eda   
https://www.kaggle.com/anshuls235/time-series-forecasting-eda-fe-modelling